In [121]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import cv2

from PIL import Image
from tensorflow import keras
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [122]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 8

In [123]:
data_path = os.path.join(os.getcwd(), 'data')
train_path = os.path.join(data_path, 'train')
valid_path = os.path.join(data_path, 'valid')
test_path = os.path.join(data_path, 'test')

In [124]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [125]:
train_generator = data_generator.flow_from_directory(train_path, 
                                                     target_size=IMAGE_SIZE, 
                                                     batch_size=BATCH_SIZE, 
                                                     class_mode='categorical')

valid_generator = data_generator.flow_from_directory(valid_path, 
                                                     target_size=IMAGE_SIZE, 
                                                     batch_size=BATCH_SIZE, 
                                                     class_mode='categorical')

Found 300 images belonging to 2 classes.
Found 103 images belonging to 2 classes.


In [86]:
base_model = tf.keras.applications.ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
pred_layer = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.Sequential()

model.add(base_model)
model.add(global_average_layer)
# model.add(tf.keras.layers.Dense(256, activation='relu'))
# model.add(tf.keras.layers.Dropout(.25))
# model.add(tf.keras.layers.BatchNormalization())
model.add(pred_layer)
#Freeze the resnet layer
model.layers[0].trainable = False
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-2), 
              loss='binary_crossentropy', 
              metrics= ['accuracy']
             )

In [87]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_9 ( (None, 2048)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 256)               524544    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 256)               1024      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 257       
Total params: 24,090,625
Trainable params: 525,313
Non-trainable params: 23,565,312
____________________________________

In [88]:
train_steps = len(train_generator)/BATCH_SIZE
valid_steps = len(valid_generator)/BATCH_SIZE
model_save_path = os.path.join(data_path, 'models', 'tf_resnet50v2.h5')
early_stop = EarlyStopping(monitor= 'val_loss', 
                           patience = 10)
model_checkpoint = ModelCheckpoint(filepath=model_save_path, 
                                   monitor='val_loss', 
                                   save_best_only=True)

In [89]:
fit_history = model.fit_generator(
              train_generator,
              steps_per_epoch=train_steps,
              epochs = 20,
              validation_data=valid_generator,
              validation_steps=valid_steps,
              callbacks=[model_checkpoint])

Epoch 1/20
5/4 [===============================] - 2s 461ms/step - loss: 0.8738 - accuracy: 0.5000 - val_loss: 0.9205 - val_accuracy: 0.5000
Epoch 2/20
5/4 [===============================] - 2s 359ms/step - loss: 0.8583 - accuracy: 0.5000 - val_loss: 0.8729 - val_accuracy: 0.5000
Epoch 3/20
5/4 [===============================] - 2s 322ms/step - loss: 0.8311 - accuracy: 0.5000 - val_loss: 0.9466 - val_accuracy: 0.5000
Epoch 4/20
5/4 [===============================] - 2s 309ms/step - loss: 0.8509 - accuracy: 0.5000 - val_loss: 0.8865 - val_accuracy: 0.5000
Epoch 5/20
2/4 [===========>..................] - ETA: 0s - loss: 0.7936 - accuracy: 0.5000

KeyboardInterrupt: 

In [16]:
img_ext = ('png', 'jpg', 'jpeg', 'tiff', 'bmp', 'gif')
test_img_list = [os.path.join(test_path, img) for img in os.listdir(test_path) if img.endswith(('png', 'jpg', 'jpeg', 'tiff', 'bmp', 'gif'))]
test_img_list

['C:\\Users\\nzahi\\PycharmProjects\\DataAnalysis\\Smile Detector\\data\\test\\nosmile1.jpg',
 'C:\\Users\\nzahi\\PycharmProjects\\DataAnalysis\\Smile Detector\\data\\test\\nosmile10.jpg',
 'C:\\Users\\nzahi\\PycharmProjects\\DataAnalysis\\Smile Detector\\data\\test\\nosmile2.jpg',
 'C:\\Users\\nzahi\\PycharmProjects\\DataAnalysis\\Smile Detector\\data\\test\\nosmile3.jpg',
 'C:\\Users\\nzahi\\PycharmProjects\\DataAnalysis\\Smile Detector\\data\\test\\nosmile4.jpg',
 'C:\\Users\\nzahi\\PycharmProjects\\DataAnalysis\\Smile Detector\\data\\test\\nosmile5.jpg',
 'C:\\Users\\nzahi\\PycharmProjects\\DataAnalysis\\Smile Detector\\data\\test\\nosmile6.jpg',
 'C:\\Users\\nzahi\\PycharmProjects\\DataAnalysis\\Smile Detector\\data\\test\\nosmile7.jpg',
 'C:\\Users\\nzahi\\PycharmProjects\\DataAnalysis\\Smile Detector\\data\\test\\nosmile8.jpg',
 'C:\\Users\\nzahi\\PycharmProjects\\DataAnalysis\\Smile Detector\\data\\test\\nosmile9.jpg',
 'C:\\Users\\nzahi\\PycharmProjects\\DataAnalysis\\Smile De

In [113]:
def preprocess_image(filepath):
    im = image.load_img(test_img_1, target_size=IMAGE_SIZE)
    im_arr = np.array(im)
    im.close()
    pre_process_img = preprocess_input(im_arr)
    pre_process_img = np.expand_dims(pre_process_img, axis=0)
    return pre_process_img

In [107]:
test_img_1 = test_img_list[0]
im = image.load_img(test_img_1, target_size=IMAGE_SIZE)
im_arr = np.array(im)
test = preprocess_input(im_arr)
a = np.expand_dims(test, axis=0)
a.shape

In [117]:
test_images = []
for img in test_img_list:
    test_img = preprocess_image(img)
    test_images.append(test_img)

In [118]:
preds = []
for test_img in test_images:
    pred = model.predict_classes(test_img)
    preds.append(*pred)
    

In [119]:
preds

[array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1])]